In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from owi_data_2_pandas.io import API
import datetime
from pytz import utc
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Import vub Meetenet Vlaamse Banken API functions
from vubmvbc.config import Credentials
from vubmvbc.client import Base
from vubmvbc.objects import Catalog, Data
import vubmvbc.data_getter as dg

from oma_tracking.oma_clustering import ModeClusterer  #pip install hdbscan --no-build-isolation --no-binary :all
from oma_tracking.data.make_dataset import DatasetGetter

In [7]:
user = os.getenv('API_USERNAME') 
password = os.getenv('API_PASSWORD')
root = r"http://10.0.0.138/api/v1/"
owi = API(api_root=root, username=user, password=password)

In [9]:
owi.groups.iloc[2]

project                         parkwind
locations                        [bbc01]
datagroups    [acceleration, mpe, scada]
Name: 3, dtype: object

In [35]:
start  = datetime.datetime(2014,1,1,tzinfo=utc)
stop  = datetime.datetime(2023,7,19,tzinfo=utc)

location = 'bbc01'
name_location = 'BB_C01'

dsg = DatasetGetter(start, stop, location)
dsg.get_longterm_data()

In [11]:
home_folder = "../../"
# Data Paths
data_file_name = '_'.join([location, start.strftime("%Y%m%d"), stop.strftime("%Y%m%d")])
data_path = home_folder + "data/bb/raw/" + data_file_name + ".parquet"
dsg.save_data(data_path)

In [33]:
def unpack_mode(data, mode = 'SS1'):
    df = data.filter(regex='SS1')
    mode = pd.DataFrame.from_records(df[df.columns[0]].explode().dropna().tolist(), index=df[df.columns[0]].explode().dropna().index)
    return mode
def get_frequencies(data, mode = 'SS1'):
    mode = unpack_mode(data, mode)
    frequency = mode.filter(regex='mean_frequency')
    return frequency

In [34]:
SS1 = get_frequencies(dsg.data, 'SS1')
SS2 = get_frequencies(dsg.data, 'SS2')
FA1 = get_frequencies(dsg.data, 'FA1')